In [1]:
import json
import pandas as pd
import numpy as np
with open('C:\\Users\\todosuk\\Desktop\\shipsnet.json') as data_file:
    dataset = json.load(data_file)
data = pd.DataFrame(dataset)
data.head(10)

,data,labels,locations,scene_ids
0,"[41, 40, 41, 44, 43, 40, 38, 40, 44, 42, 31, 3...",1,"[-122.34270462076896, 37.75058641760489]",20170501_180618_1005
1,"[22, 19, 26, 26, 21, 20, 26, 24, 24, 28, 20, 1...",1,"[-122.34975593269652, 37.76979644106731]",20170501_181320_0e1f
2,"[112, 112, 106, 104, 107, 109, 113, 115, 108, ...",1,"[-122.34908354683306, 37.74935096766827]",20170609_180756_103a
3,"[72, 76, 73, 72, 73, 74, 75, 73, 74, 73, 73, 7...",1,"[-122.32613080820607, 37.73816285249999]",20170721_180825_100b
4,"[63, 58, 58, 63, 65, 66, 70, 66, 59, 67, 71, 6...",1,"[-122.3332290637038, 37.721168346437786]",20170702_180943_103c
5,"[136, 136, 132, 133, 136, 129, 126, 128, 134, ...",1,"[-122.32982392204339, 37.747728821717814]",20170609_180751_101b
6,"[99, 94, 86, 91, 99, 96, 102, 103, 95, 97, 100...",1,"[-122.33472265432373, 37.759680938376434]",20170830_181004_0f4e
7,"[77, 76, 77, 75, 70, 77, 80, 78, 76, 76, 78, 7...",1,"[-122.32284323152084, 37.71781032748524]",20170705_180816_103e
8,"[68, 75, 68, 61, 65, 73, 67, 61, 63, 67, 63, 6...",1,"[-122.32580109488852, 37.70693034167392]",20170620_180742_102f
9,"[134, 134, 136, 134, 133, 136, 137, 136, 137, ...",1,"[-122.34931029388966, 37.77011845299925]",20170803_234846_0c54


In [2]:
x = np.array(dataset['data']).astype('uint8')
y = np.array(dataset['labels']).astype('uint8')

In [3]:
print(x.shape)
print(y.shape)

(2800, 19200)
(2800,)


In [4]:
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2 ,random_state = 123)

C:\Users\todosuk\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
print(x_train.shape)
print(y_train.shape)

(2240, 19200)
(2240,)


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, cross_val_score



def compareABunchOfDifferentModelsAccuracy(a,b,c,d):   
    print('\nCompare Multiple Classifiers:')
    print('\nK-Fold Cross-Validation Accuracy:\n')
    models = []
    models.append(('LR', LogisticRegression()))
    models.append(('RF', RandomForestClassifier()))
    models.append(('KNN', KNeighborsClassifier()))
    models.append(('SVM', SVC()))
    models.append(('LSVM', LinearSVC()))
    models.append(('GNB', GaussianNB()))
    models.append(('DTC', DecisionTreeClassifier()))
    models.append(('XGB', GradientBoostingClassifier()))
  
    resultsAccuracy = []
    names = []
    for name, model in models:
        model.fit(a,b)
        kfold = KFold(n_splits=10, random_state=7)
        accuracy_results = cross_val_score(model, a,b, cv=kfold, scoring='accuracy')
        resultsAccuracy.append(accuracy_results)
        names.append(name)
        accuracyMessage = "%s: %f (%f)" % (name, accuracy_results.mean(), accuracy_results.std())
        print(accuracyMessage)
compareABunchOfDifferentModelsAccuracy(x_train,y_train,x_test,y_test)


Compare Multiple Classifiers:

K-Fold Cross-Validation Accuracy:

LR: 0.866518 (0.021820)
RF: 0.931696 (0.019870)
KNN: 0.916518 (0.014265)
SVM: 0.745536 (0.031441)
LSVM: 0.870982 (0.022002)
GNB: 0.642411 (0.024078)
DTC: 0.887500 (0.016439)
XGB: 0.944196 (0.014293)


In [7]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
alg_frst_model = GradientBoostingClassifier(random_state=1)
alg_frst_params = [{
    "n_estimators":range (100,500,100),
    "min_samples_split": range(200,1001,200),
    "min_samples_leaf": range(1, 71, 10),
    'max_depth':range(5,16,2),
    'max_features':range(7,20,2)
}]
alg_frst_grid = GridSearchCV(alg_frst_model, alg_frst_params, cv=cv, refit=True, verbose=1, n_jobs=-1)
alg_frst_grid.fit(x_train, y_train)
alg_frst_best = alg_frst_grid.best_estimator_
print("Accuracy (random forest auto): {} with params {}"
      .format(alg_frst_grid.best_score_, alg_frst_grid.best_params_))

Fitting 3 folds for each of 5880 candidates, totalling 17640 fits


[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 15.5min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 27.6min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 43.3min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 62.9min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed: 86.6min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 113.9min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed: 143.8min
[Parallel(n_jobs=-1)]: Done 4996 tasks      | elapsed: 178.6min
[Parallel(n_jobs=-1)]: Done 6046 tasks      | elapsed: 217.6min
[Parallel(n_jobs=-1)]: Done 7196 tasks      | elapsed: 257.8min
[Parallel(n_jobs=-1)]: Done 8446 tasks      | elapsed: 304.1min
[Parallel(n_jobs=-1)]: Done 9796 tasks      | elapsed: 352.3min
[Parallel(n_jobs=-1)]: Done 11246 tasks      | elapsed: 405.6min
[Parallel(n_jobs=-1)]: Done 12796 tasks      | ela

Accuracy (random forest auto): 0.9651785714285714 with params {'max_depth': 9, 'max_features': 11, 'min_samples_leaf': 21, 'min_samples_split': 200, 'n_estimators': 400}
